In [201]:
from math import sqrt
import random
import numpy as np

## Read documents

In [1]:
docs = ["Lorem Ipsum is simply dummy text of the printing and typesetting industry.",
        "Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book.",
        "It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged.",
        "It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum."]

In [167]:
class Shingling():
    def __init__(self, k):
        self.k = k
        self.hash = hash
    def transform(self, doc):
        # Compute shingles
        shingles = [doc[i:i+self.k] for i in range(0, len(doc) - self.k + 1)]
        hashes = sorted(set([self.hash(shingle) for shingle in shingles]))
        
        return hashes

In [168]:
sh = Shingling(2)
sh.transform(docs[0])[0:5]

[-8907460678534539826,
 -8750419651505896790,
 -8669368805443004484,
 -8474637928453286226,
 -8081243204084430761]

In [169]:
class CompareSets():
    def compare(self, a, b):
        return len(set(a) & set(b)) / len(set(a) | set(b))

In [80]:
cs = CompareSets()
cs.compare(sh.transform(docs[1]), sh.transform(docs[3]))

0.3388888888888889

In [236]:
class MinHashing():
    def __init__(self, k):
        primes = self._primes(k)
        self.hashes = [lambda x: (random.randint(1, k)*x + random.randint(1, k))%random.choice(primes)
                       for i in range(k)]
        
    def transform(self, doc):
        signatures = [[f(shingle) for shingle in doc] for f in self.hashes]
        return [doc[np.argmin(l)] for l in signatures]
        
        
    def _primes(self, n):
        # Function that returns the first n prime numbers
        # Based on Eratostenes sieve
        primes = [2, 3, 5, 7, 11, 13, 17, 19, 23]
        while(len(primes) < n):
            last = primes[-1]
            primes += [i for i in range(last, last*last) if all(i%p for p in primes)]
        return primes[0:n]

In [258]:
mh = MinHashing(5)
mh.transform(sh.transform(docs[1]))

[-8740365127586544774,
 -9099774106317024168,
 -9152175143030237047,
 -8665697011579506802,
 -8750419651505896790]

In [259]:
class CompareSignatures():
    def compare(self, a, b):
        if len(a) != len(b):
            raise ValueError("Signatures lengths differ.")
        return sum([int(a[i] == b[i]) for i in range(len(a))]) / len(a)

In [260]:
csign = CompareSignatures()
csign.compare(mh.transform(sh.transform(docs[1])), mh.transform(sh.transform(docs[3])))

0.2